In [18]:
# conda env: clamp_env (Python 3.8.0)
import os
import sys

import pandas as pd
import torch
import torch.nn.functional as F
import numpy 

from datacat4ml.const import *
from datacat4ml.utils import mkdirs
from datacat4ml.Scripts.model_dev.metrics import *

# clamp models can only be used for classification tasks
import clamp

# Load the test datasets

In [19]:
#def clamp_predict(test_smis, test_assay_descs):
#    """
#    Predict the bioactivity of molecules using CLAMP model.
#    """
#    # Load the model
#    model = clamp.CLAMP(device='cpu') # after runnning this file, the folder `data/models/clamp_clip' will be created and stroe the downloaded models
#    model.eval()
#
#    with torch.no_grad():
#        logits = model.forward_dense(test_smis, test_assay_descs)
#        probs = logits.softmax(dim=0).cpu.numpy()  # probs for molecucles
#    
#    return probs

In [20]:
for use_clustering in Use_clusterings:

    print (f'---> Use clustering: {use_clustering}')

    folder_path = os.path.join(SPLIT_CAT_DATASETS_DIR, 'cls', 'use_clustering'+'_'+str(use_clustering))
    files = os.listdir(folder_path)
    split_files = [file for file in files if file.endswith('splited.csv')]

    for split_file in split_files:
        
        print(f'Take the split file: {split_file}\n')
        split_file_path = os.path.join(folder_path, split_file)
        df = pd.read_csv(split_file_path)

        # get the smiles and assay descriptions in the test set
        test_smis = df[df['split'] == 'test']['canonical_smiles_by_Std'].tolist()
        print(f'The length of test_smis is {len(test_smis)}')

        test_assay_descs = df[df['split'] == 'test']['assay_desc'].tolist()
        print(f'\nThe length of test_assay_descs is {len(test_assay_descs)}')

        # load the clamp model
        model = clamp.CLAMP(device='cpu') # after runnning this file, the folder `data/models/clamp_clip' will be created and stroe the downloaded models
        model.eval()

        # calculate the probabilities of the test set
        with torch.no_grad():
            logits = model.forward_dense(test_smis, test_assay_descs)
            probs = logits.softmax(dim=0).cpu().numpy()

        # get the predictions based on the probabilities
        probs_list = []
        pred_list = []
        for i in range(len(probs)):
            probs_list.append(probs[i][i])
            if probs[i][i] > 0.5:
                pred_list.append(1)
            else:
                pred_list.append(0)
        print(f'probs_list: {probs_list}')
        print(f'pred_list: {pred_list}')
        
        # get the true labels
        true_list = df[df['split'] == 'test']['activity'].tolist()
        print(f'true_list: {true_list}')   

        # calculate the metrics
        accuracy = calc_accuracy(true_list, pred_list)
        precision = calc_precision(true_list, pred_list)
        recall = calc_recall(true_list, pred_list)
        # roc_auc = calc_roc_auc(true_list, probs_list)
        # f1 = calc_f1(true_list, pred_list)
        mcc = calc_mcc(true_list, pred_list)
        
        #bedroc_dec5 = bedroc_score(true_list, pred_list, alpha=321.9)
        #bedroc_2 = bedroc_score(true_list, pred_list, alpha=80.5)
        #bedroc_8 = bedroc_score(true_list, pred_list, alpha=20.0)

        # columns to be written to the results file
        file_path_name = 'CAT_ORs'
        task = 'cls'
        use_smote = 'None'
        target = df['target_chembl_id'].iloc[0]
        effect = df['effect'].iloc[0]
        assay = df['assay'].iloc[0]
        std_type = df['std_type'].iloc[0]
        descriptor = 'FP'
        algoname = 'CLAMP'
        n_compounds = len(df)
        n_cliff_compounds = 'NA'
        n_compounds_train = len(df[df['split'] == 'train'])
        n_cliff_compounds_train = 'NA'
        n_compounds_test = len(df[df['split'] == 'test'])
        n_cliff_compounds_test = 'NA'
        threshold = df['threshold'].iloc[0]
        rmse = 'None'
        cliff_rmse = 'None'
        r2 = 'None'
        cliff_r2 = 'None'

        # temp
        bedroc_dec5 = 'None'
        bedroc_2 = 'None'
        bedroc_8 = 'None'
        roc_auc = 'None'
        f1 = 'None'

        if not os.path.isfile(os.path.join(BM_CAT_DIR , 'results.csv')):
            with open(os.path.join(BM_CAT_DIR , 'results.csv'), 'w') as f:
                f.write('file_path,task,use_clustering,use_smote,'
                        'target,effect,assay,std_type,descriptor,algo,'
                        'n_compounds,n_cliff_compounds,n_compounds_train,n_cliff_compounds_train,n_compounds_test,n_cliff_compounds_test,'
                        'threshold,'
                        'accuracy, precision, recall, mcc,bedroc_dec5,bedroc_2,bedroc_8,'
                        'rmse,cliff_rmse,r2,cliff_r2\n')
        
        with open(os.path.join(BM_CAT_DIR , 'results.csv'), 'a') as f:
            f.write(f'{file_path_name},{task},{use_clustering},{use_smote},'
                    f'{target},{effect},{assay},{std_type},{descriptor},{algoname},'
                    f'{n_compounds},{n_cliff_compounds},{n_compounds_train},{n_cliff_compounds_train},{n_compounds_test},{n_cliff_compounds_test},'
                    f'{threshold},' 
                    f'{accuracy},{precision},{recall},{mcc},{bedroc_dec5},{bedroc_2},{bedroc_8},'
                    f'{rmse},{cliff_rmse},{r2},{cliff_r2} \n')

---> Use clustering: True
Take the split file: dor_agon_G_cAMP_EC50_splited.csv

The length of test_smis is 27

The length of test_assay_descs is 27


2024-11-17 19:25:16.532 | INFO     | clamp.models.pretrained:__init__:33 - Loaded pretrained model from data/models/clamp_clip/checkpoint.pt


probs_list: [0.03853938, 0.038883094, 0.038946893, 0.03751956, 0.034175284, 0.03624074, 0.03683666, 0.035696123, 0.03718598, 0.03563619, 0.034990437, 0.03914062, 0.037474, 0.036945995, 0.036367416, 0.036320467, 0.036863495, 0.036010496, 0.036276735, 0.03506068, 0.035552606, 0.03922239, 0.038987536, 0.035330664, 0.035542544, 0.037464354, 0.03527097]
pred_list: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
true_list: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Take the split file: nor_agon_G_cAMP_IC50_splited.csv

The length of test_smis is 16

The length of test_assay_descs is 16


/storage/homefs/yc24j783/miniconda3/envs/clamp_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2024-11-17 19:25:19.533 | INFO     | clamp.models.pretrained:__init__:33 - Loaded pretrained model from data/models/clamp_clip/checkpoint.pt


probs_list: [0.066501416, 0.06652624, 0.066646904, 0.061422948, 0.06028056, 0.061658725, 0.059790105, 0.0613236, 0.05956159, 0.059954207, 0.066479124, 0.06869329, 0.061130527, 0.058447275, 0.057759438, 0.06557117]
pred_list: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
true_list: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
Take the split file: dor_antag_G_GTP_IC50_splited.csv

The length of test_smis is 35

The length of test_assay_descs is 35


...

/storage/homefs/yc24j783/miniconda3/envs/clamp_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2024-11-17 19:29:36.180 | INFO     | clamp.models.pretrained:__init__:33 - Loaded pretrained model from data/models/clamp_clip/checkpoint.pt


probs_list: [0.011954536, 0.012498848, 0.012250828, 0.012321637, 0.012121629, 0.012494718, 0.012184513, 0.012475473, 0.013116063, 0.011322965, 0.013268965, 0.011609777, 0.012724337, 0.012004706, 0.011441384, 0.013091257, 0.01146475, 0.0112017635, 0.014124026, 0.012094121, 0.011840117, 0.011568363, 0.0114545375, 0.012323343, 0.011189438, 0.011651524, 0.011445156, 0.011715759, 0.011608365, 0.0112893805, 0.011896715, 0.010935305, 0.011863769, 0.01218972, 0.011718872, 0.011963392, 0.012440575, 0.012603053, 0.012354353, 0.012531363, 0.011812665, 0.011660416, 0.012569014, 0.011737526, 0.012456343, 0.011872093, 0.011810679, 0.012495746, 0.011878275, 0.011604128, 0.012275886, 0.012184235, 0.012388788, 0.012604217, 0.011987327, 0.012030153, 0.011794326, 0.01167624, 0.012490803, 0.012677002, 0.010770695, 0.012024785, 0.011935531, 0.0118767, 0.011831732, 0.011464607, 0.012958579, 0.010998707, 0.010818032, 0.0107944785, 0.010543151, 0.010583081, 0.010493277, 0.010410494, 0.012743688, 0.012986704, 

/storage/homefs/yc24j783/miniconda3/envs/clamp_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2024-11-17 19:29:39.836 | INFO     | clamp.models.pretrained:__init__:33 - Loaded pretrained model from data/models/clamp_clip/checkpoint.pt


probs_list: [0.004025487, 0.0041861786, 0.0040902337, 0.00395572, 0.0038928005, 0.0038043738, 0.004398653, 0.004077139, 0.004191009, 0.0032617194, 0.003242239, 0.0040375525, 0.00394235, 0.004081597, 0.0037373896, 0.004242307, 0.004008215, 0.0035421378, 0.0034426313, 0.0033138301, 0.0033612214, 0.004400866, 0.0034590045, 0.0037065444, 0.004149054, 0.0039966735, 0.0041785506, 0.0034383324, 0.0033847368, 0.0035450764, 0.003433226, 0.0034809988, 0.0034702863, 0.0043308227, 0.003967316, 0.0041204095, 0.004109438, 0.004024578, 0.004006741, 0.003946531, 0.003932907, 0.004004164, 0.003963192, 0.0039087, 0.0038994937, 0.004029326, 0.004211832, 0.0034065382, 0.003446041, 0.0033799126, 0.0034404658, 0.0038584322, 0.003755722, 0.004360006, 0.0045068786, 0.004484784, 0.0044888817, 0.0037754087, 0.004420746, 0.0041384595, 0.00424966, 0.0042029913, 0.0041500973, 0.004069666, 0.0040418697, 0.0037871846, 0.0038517169, 0.0037782162, 0.0040990156, 0.0041168677, 0.0036346798, 0.0036774916, 0.0035538038, 0

/storage/homefs/yc24j783/miniconda3/envs/clamp_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
